In [2]:
import numpy as np
import pickle
import cv2
import tensorflow as tf
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import load_model
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
np.set_printoptions(precision = 3, suppress = True)

#from tensorflow.keras import  layers
#from tensorflow.keras.layers.experimental import preprocessing

In [26]:
print("Num of Gpus avaialble:", len(tf.config.experimental.list_physical_devices('GPU')))
#GTX_1060 = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(GTX_1060[0], True)

Num of Gpus avaialble: 0


## Defining Model

In [37]:
model = Sequential([
        Dense(units = 96, input_shape=(44,), activation = 'relu'),
        Dense(units = 64, activation='relu'),
        Dense(units = 32, activation='relu'),
        Dense(units = 24, activation='relu'),
        Dense(units = 12, activation ='relu'),
        Dense(units = 3, activation='softmax')
])

In [41]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 96)                4320      
_________________________________________________________________
dense_25 (Dense)             (None, 64)                6208      
_________________________________________________________________
dense_26 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_27 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_28 (Dense)             (None, 12)                300       
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 39        
Total params: 13,739
Trainable params: 13,739
Non-trainable params: 0
__________________________________________________

## Loading the dataset

In [10]:
train_dataset = pd.read_csv("C:/Users/Praty/OneDrive/Desktop/train.csv")
test_dataset = pd.read_csv("C:/Users/Praty/OneDrive/Desktop/test.csv")

In [11]:
train_dataset.head()

,Unnamed: 0,MEAN_RR,MEDIAN_RR,RMSSD,SDSD,HR,pNN25,SD1,SKEW,MEDIAN_REL_RR,...,SD1_LOG,SKEW_YEO_JONSON,SKEW_REL_RR_YEO_JONSON,HF_BOXCOX,HR_SQRT,HR_LF,HR_HF,HF_VLF,SSSQ Label,condition label
0,0,860.758450,858.389130,14.461513,14.460581,70.242263,6.25,10.238012,0.053677,-0.000286,...,2.419302,0.053188,0.053188,3.307810,8.381066,0.091228,3.666502,0.009514,0,0
1,1,1044.449374,1036.796500,22.516679,22.515843,57.792396,28.50,15.941094,0.578705,-0.000638,...,2.829742,0.671867,0.671867,0.871954,7.602131,0.030044,25.537614,0.001175,0,0
2,2,510.072193,503.341590,8.549018,8.547740,118.059988,3.50,6.051753,1.623759,0.000003,...,1.953276,2.004587,2.004587,7.869055,10.865541,0.333489,3.031509,0.084238,1,2
3,3,930.296969,940.394165,7.816167,7.814190,64.769049,0.00,5.532404,-0.596994,-0.000397,...,1.876775,-0.636446,-0.636446,1.999268,8.047922,0.392480,11.410422,0.004625,2,0
4,4,692.067906,687.546285,6.487698,6.487256,87.096547,0.00,4.592942,0.491406,0.001556,...,1.721505,0.465961,0.465961,1.686187,9.332553,0.322039,19.549568,0.003409,2,2


In [12]:
test_dataset.head()

,Unnamed: 0,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,...,MEAN_RR_MEAN_MEAN_REL_RR,SD2_LF,HR_LF,HR_HF,HF_VLF,subject id,condition,SSSQ class,SSSQ Label,condition label
0,0,856.640559,855.196440,63.543133,15.490732,15.490690,4.102010,70.436793,9.75,0.0,...,-1.241198e+07,0.083708,0.066106,2.156205,0.031908,8,baseline,medium,1,0
1,1,911.890969,914.313935,49.739529,11.485676,11.485584,4.330570,65.996032,3.50,0.0,...,3.087452e+07,0.142285,0.134394,9.215434,0.005218,6,baseline,medium,1,0
2,2,505.424097,500.162060,27.207534,5.217502,5.216857,5.214666,119.049905,0.00,0.0,...,-4.138619e+06,0.180380,0.560690,8.795038,0.072400,11,stress,high,2,2
3,3,976.606489,976.392520,93.145519,24.999690,24.999508,3.725867,62.014311,32.25,4.0,...,-8.913369e+06,0.058028,0.027568,9.998307,0.002822,17,baseline,low,0,0
4,4,740.323476,729.095405,73.676039,10.740166,10.739891,6.859860,81.833707,3.75,0.0,...,-5.150339e+06,0.193675,0.152519,5.783011,0.004499,15,baseline,medium,1,0


In [13]:
header_list = train_dataset.columns.values

In [29]:
df = train_dataset[header_list[1:46]]

In [30]:
X = df.iloc[:,:44]
Y = train_dataset[['condition label']]

In [16]:
X

,MEAN_RR,MEDIAN_RR,RMSSD,SDSD,HR,pNN25,SD1,SKEW,MEDIAN_REL_RR,RMSSD_REL_RR,...,pNN25_LOG,SD1_LOG,SKEW_YEO_JONSON,SKEW_REL_RR_YEO_JONSON,HF_BOXCOX,HR_SQRT,HR_LF,HR_HF,HF_VLF,SSSQ Label
0,856.640559,855.196440,15.490732,15.490690,70.436793,9.75,10.967324,-0.102585,-0.002450,0.009186,...,2.374906,2.482180,-0.101676,-0.101676,7.769052,8.392663,0.066106,2.156205,0.031908,1
1,911.890969,914.313935,11.485676,11.485584,65.996032,3.50,8.131731,-0.178860,-0.000799,0.005881,...,1.504077,2.211755,-0.175197,-0.175197,2.359828,8.123794,0.134394,9.215434,0.005218,1
2,505.424097,500.162060,5.217502,5.216857,119.049905,0.00,3.693507,0.431171,0.000281,0.008260,...,0.000000,1.546180,0.434722,0.434722,3.584964,10.910999,0.560690,8.795038,0.072400,2
3,976.606489,976.392520,24.999690,24.999508,62.014311,32.25,17.699516,-0.127575,0.002974,0.011375,...,3.504055,2.928498,-0.130049,-0.130049,1.838114,7.874917,0.027568,9.998307,0.002822,0
4,740.323476,729.095405,10.740166,10.739891,81.833707,3.75,7.603784,0.402807,0.001584,0.006013,...,1.558145,2.152202,0.411162,0.411162,3.258883,9.046198,0.152519,5.783011,0.004499,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13560,823.312502,822.335095,15.580839,15.579649,73.177890,8.50,11.030307,0.121589,-0.003110,0.009085,...,2.251292,2.487429,0.121897,0.121897,4.678582,8.554408,0.058068,3.095154,0.029057,0
13561,813.487251,812.393075,11.722249,11.719850,73.905611,3.50,8.297590,-0.148241,-0.000644,0.009398,...,1.504077,2.229755,-0.153921,-0.153921,9.108968,8.596837,0.194929,1.471170,0.171440,2
13562,1038.999403,1031.351750,26.508801,26.507248,58.174854,34.75,18.766987,0.475746,-0.000789,0.011510,...,3.576550,2.984013,0.539390,0.539390,1.672609,7.627244,0.023563,13.195805,0.001831,0
13563,716.291740,714.843350,5.528628,5.522179,84.037013,0.00,3.909673,0.441007,-0.000107,0.003343,...,0.000000,1.591207,0.420185,0.420185,2.233718,9.167170,0.794870,12.422019,0.011508,2


In [17]:
Y

,condition label
0,0
1,0
2,2
3,0
4,0
...,...
13560,0
13561,0
13562,0
13563,2


In [39]:
model.compile(optimizer = 'Adam', loss ='sparse_categorical_crossentropy', metrics=['accuracy'])

In [40]:
model.fit(x = X, y = Y, batch_size = 10, epochs = 120, verbose =2)

Epoch 1/120
10852/10852 - 9s - loss: 0.8734 - accuracy: 0.7126
Epoch 2/120
10852/10852 - 9s - loss: 0.4755 - accuracy: 0.7980
Epoch 3/120
10852/10852 - 9s - loss: 0.3549 - accuracy: 0.8502
Epoch 4/120
10852/10852 - 9s - loss: 0.3075 - accuracy: 0.8737
Epoch 5/120
10852/10852 - 8s - loss: 0.2792 - accuracy: 0.8862
Epoch 6/120
10852/10852 - 8s - loss: 0.2605 - accuracy: 0.8943
Epoch 7/120
10852/10852 - 8s - loss: 0.2471 - accuracy: 0.8992
Epoch 8/120
10852/10852 - 8s - loss: 0.2316 - accuracy: 0.9064
Epoch 9/120
10852/10852 - 9s - loss: 0.2201 - accuracy: 0.9112
Epoch 10/120
10852/10852 - 8s - loss: 0.2171 - accuracy: 0.9124
Epoch 11/120
10852/10852 - 8s - loss: 0.2090 - accuracy: 0.9168
Epoch 12/120
10852/10852 - 9s - loss: 0.1979 - accuracy: 0.9203
Epoch 13/120
10852/10852 - 9s - loss: 0.1886 - accuracy: 0.9252
Epoch 14/120
10852/10852 - 9s - loss: 0.1794 - accuracy: 0.9295
Epoch 15/120
10852/10852 - 9s - loss: 0.1772 - accuracy: 0.9304
Epoch 16/120
10852/10852 - 9s - loss: 0.1672 - ac

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7159470482844981096
]


In [ ]:
print(tf.__version__)